In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import torchvision.datasets as datasets

import torch.nn as nn
import torch.optim as optim

import torch
from torchvision import transforms
from torchvision.datasets import SVHN

In [ ]:
mean_normalize = [0.491, 0.482, 0.456]
std_dev_normalize = [0.229, 0.224, 0.225]
crop_size= 224
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_normalize,std_dev_normalize )
])

In [ ]:
img_dataset = ImageFolder('/kaggle/input/cnn-dataset', 
                          transform=data_transform)


In [ ]:
data_split_ratio = 0.8
img_set_size = len(img_dataset)
train_ratio = int(img_set_size * data_split_ratio)
test_ratio = int(img_set_size - train_ratio)
train_data, test_data = torch.utils.data.random_split(img_dataset,
                                                      [train_ratio, test_ratio])
batch_size = 64
train_img_loader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=batch_size,
                                               shuffle=True)
test_img_loader = torch.utils.data.DataLoader(test_data, 
                                              batch_size=batch_size, 
                                              shuffle=False)



In [ ]:
##some sample train images
class_labels = (1, 2, 0)
for idx, batch in enumerate(train_img_loader):
    fig, axs = plt.subplots(1, 5, figsize=(10, 5))
    inputs = batch[0]
    labels = batch[1]
    for j in range(0,5):
        axs[j].imshow(inputs[j].permute(class_labels)),axs[j].axis('off'),axs[j].set_title(f'Label: {labels[j]}')
    plt.show()
    break


In [ ]:
class_labels = (1, 2, 0)
for idx, batch in enumerate(test_img_loader): ##similarly lets checkout some test samples
    fig, axs = plt.subplots(1, 5, figsize=(10, 5))
    inputs = batch[0]
    labels = batch[1]
    for j in range(0,5):
        axs[j].imshow(inputs[j].permute(class_labels)),axs[j].axis('off'),axs[j].set_title(f'Label: {labels[j]}')
    plt.show()
    break
    

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=3):
        super(AlexNet, self).__init__()
        self.num_classes = num_classes
        self.localnorm = nn.LocalResponseNorm(size=5)
        self.layer1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
        self.activation_layer_relu = nn.ReLU(inplace=True)
        self.max_pool_layer = nn.MaxPool2d(kernel_size=3, stride=2)
        self.layer2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.layer5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.layer6 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropuput_layer = nn.Dropout(0.5)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

        # initialize weights
        self.apply(self.initialize_weights)

    def forward(self, x):
        # move tensors to CUDA device
        x = x.cuda()
        out = self.layer1(x)
        out = self.activation_layer_relu(out)
        out = self.localnorm(out)
        out = self.max_pool_layer(out)
        out = self.layer2(out)
        out = self.activation_layer_relu(out)
        out = self.localnorm(out)
        out = self.max_pool_layer(out)
        out = self.layer3(out)
        out = self.activation_layer_relu(out)
        out = self.layer4(out)
        out = self.activation_layer_relu(out)
        out = self.layer5(out)
        out = self.activation_layer_relu(out)
        out = self.layer6(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.activation_layer_relu(out)
        out = self.dropuput_layer(out)
        out = self.fc2(out)
        out = self.activation_layer_relu(out)
        out = self.dropuput_layer(out)
        out = self.fc3(out)
        return out

    def initialize_weights(self, m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Availabe device: ", device)

In [ ]:
learning_rate = 0.001
momentum_val = 0.9
model = AlexNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum_val)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(0,50):
    train_loss, train_correct, total_train = 0.0, 0, 0

    model.train()
    for i, batch in enumerate(train_img_loader):
        ip, labl = batch
        optimizer.zero_grad()
        ip, labl = ip.to("cuda"), labl.to("cuda")  ## move the data to GPU
        outputs = model(ip)
        loss = criterion(outputs, labl)
         
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        opt, y_hat = torch.max(outputs.data, 1)
        total_correct_val = (y_hat == labl)
        train_correct += total_correct_val.sum().item()
        total_train += labl.size(0)

    test_loss, test_correct, test_total = 0.0, 0, 0

    model.eval()

    with torch.no_grad():
        for i, batch in enumerate(test_img_loader):
            ip, labl = batch
            ip, labl = ip.to("cuda"), labl.to("cuda")  ## move the data to GPU
            outputs = model(ip)
            loss = criterion(outputs, labl)
            
            opt, y_hat = torch.max(outputs.data, 1)
            total_correct_val_y = (y_hat == labl)
            test_correct += total_correct_val_y.sum().item()
            test_total += labl.size(0)
            test_loss += loss.item()
    
    train_accuracy = 100 * train_correct / total_train
    train_loss /= len(train_img_loader)
    test_accuracy = 100 * test_correct / test_total
    test_loss /= len(test_img_loader)

    # Print training and test accuracy and loss for each epoch
    print(f'Currently running {epoch+1} epoch current train Loss: {train_loss:.5f} Current train Accuracy: {train_accuracy:.2f}% Our Test loss  {test_loss:.5f} And our test Accuracy: {test_accuracy:.2f}%')
